<a href="https://colab.research.google.com/github/Nemczek/chess_nerual_network/blob/main/chess_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-chess
!pip install zstandard

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 16.6 MB/s eta 0:00:00


# Gra w szachy

In [ ]:
from IPython.display import display, HTML, clear_output
import chess
import chess.svg
from ipywidgets import Button, Layout
from ipywidgets import Button, HBox, VBox, Layout

def display_board(board):
    return chess.svg.board(board=board, size=400)

def make_move(move):
    if board.is_legal(move):
        board.push(move)
        clear_output(wait=True)
        display(HTML(display_board(board)))
        display_board_buttons()

def on_button_click(b):
    san = b.description
    move = chess.Move.from_uci(board.parse_san(san).uci())
    make_move(move)

def display_board_buttons():
    legal_moves = [board.san(move) for move in board.legal_moves]
    buttons = [Button(description=san, layout=Layout(width='80px', height='40px')) for san in legal_moves]

    for button in buttons:
        button.on_click(on_button_click)

    display(VBox([HBox(buttons[:8]), HBox(buttons[8:16]), HBox(buttons[16:24]), HBox(buttons[24:32])]))

board = chess.Board()
display(HTML(display_board(board)))
display_board_buttons()

# Sieć neuronowa

## Generator danych

Ponieważ dane to ok. 40 milionów gier standardowych to byłby problem z wczytaniem ich wszystkich naraz do pamięci w celu uczenia sieci. Aby obejść ten problem możemy zastosować generator, który będzie ładował gry w batach po 32 gry.

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
chess_data = "drive/MyDrive/dane_szachy/chess_data.pgn.zst"
#chess_data = "test_game.pgn.zst"

In [ ]:
import zstandard as zstd
import io
import numpy as np
import tensorflow as tf
import chess.pgn

Funkcja mapująca figurze liczbę

In [ ]:
def get_piece_type(board, square):
    piece = board.piece_at(square)
    if piece is None:
        return 0
    piece_type = piece.piece_type
    if piece_type == chess.PAWN:
        return 1
    elif piece_type == chess.KNIGHT:
        return 2
    elif piece_type == chess.BISHOP:
        return 3
    elif piece_type == chess.ROOK:
        return 4
    elif piece_type == chess.QUEEN:
        return 5
    elif piece_type == chess.KING:
        return 6
    else:
        return 0

## Encoding gier

In [ ]:
import numpy as np
import zstandard as zstd
import io
import chess.pgn

def data_generator(path, batch_size=32, maxlen=50):
    dec_obj = zstd.ZstdDecompressor()

    x_batch = []
    y_batch = []

    with open(path, "rb") as file:
        stream = dec_obj.stream_reader(file)
        text = io.TextIOWrapper(stream, encoding="utf-8")

        while True:
              game = chess.pgn.read_game(text)
              if game is None:
                  break  # Koniec gier, zatrzymujemy generator

              moves = []
              node = game
              while node.variations:
                  next_node = node.variation(0)
                  moves.append(next_node.move.uci())
                  node = next_node
              moves = moves[:maxlen]
              if len(moves) < maxlen:
                  moves += [''] * (maxlen - len(moves))

              number_moves = []
              for move in moves:
                  if move:
                    # Odejmowane ord("a") oraz 1 sprawia, że otrzymane liczby
                    # zaczynają się od 0
                      start_col = ord(move[0]) - ord("a")
                      start_row = int(move[1]) - 1
                      end_col = ord(move[2]) - ord("a")
                      end_row = int(move[3]) - 1

                      # Sprawdza, czy ruch zakończył się promocją pionka
                      promotion = ord(move[4]) - ord('a') if len(move) == 5 else 0

                      # Ustawia special_move odpowiednio w zależności od tego
                      # Czy i jaki specjalny ruch był wykonany
                      special_move = 0

                      # Roszady
                      if move == 'e1g1' or move == 'e8g8':
                          special_move = 1
                      elif move == 'e1c1' or move == 'e8c8':
                          special_move = 2

                      # Bicie en-passant
                      elif game.board().is_en_passant(chess.Move.from_uci(move)):
                          special_move = 3

                      # Sprawdzani figury
                      piece_type = get_piece_type(game.board(), chess.square(start_col, start_row))

                      number_moves.append([start_col, start_row, end_col, end_row, promotion, special_move, piece_type])
                  else:
                      number_moves.append([0, 0, 0, 0, 0, 0, 0])

              targets = number_moves[1:] + [[0, 0, 0, 0, 0, 0, 0]]

              x_batch.append(number_moves)
              y_batch.append(targets)

              if len(x_batch) >= batch_size:
                  yield np.array(x_batch), np.array(y_batch)
                  x_batch = []
                  y_batch = []

    if x_batch and y_batch:
        yield np.array(x_batch), np.array(y_batch)

# Utworzenie generatora danych
train_generator = data_generator(chess_data, batch_size=32)

In [ ]:
print(next(train_generator)[0][0][0])

[4 1 4 3 0 0 1]


Wynikowy tensor (np e4e5) = [start_col(e), start_row(4), end_col(e), end_row(5),
czy była promocja, ruch specjalny (roszada/en_passant), jaka figura]

In [ ]:
next(train_generator)

(array([[[4, 1, 4, ..., 0, 0, 1],
         [4, 6, 4, ..., 0, 0, 1],
         [3, 1, 3, ..., 0, 0, 1],
         ...,
         [3, 7, 3, ..., 0, 0, 5],
         [7, 2, 7, ..., 0, 0, 0],
         [4, 3, 6, ..., 0, 0, 0]],
 
        [[4, 1, 4, ..., 0, 0, 1],
         [3, 6, 3, ..., 0, 0, 1],
         [3, 1, 3, ..., 0, 0, 1],
         ...,
         [0, 6, 0, ..., 0, 0, 1],
         [5, 3, 4, ..., 0, 0, 0],
         [5, 5, 4, ..., 0, 0, 0]],
 
        [[3, 1, 3, ..., 0, 0, 1],
         [4, 6, 4, ..., 0, 0, 1],
         [2, 1, 2, ..., 0, 0, 1],
         ...,
         [0, 7, 3, ..., 0, 0, 4],
         [4, 1, 4, ..., 0, 0, 1],
         [5, 5, 5, ..., 0, 0, 0]],
 
        ...,
 
        [[4, 1, 4, ..., 0, 0, 1],
         [4, 6, 4, ..., 0, 0, 1],
         [6, 0, 5, ..., 0, 0, 2],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],
 
        [[4, 1, 4, ..., 0, 0, 1],
         [3, 6, 3, ..., 0, 0, 1],
         [3, 1, 3, ..., 0, 0, 1],